This code iteratively solves for the correct ride and roll rate using the method described in section 16 of RCVD. It solves for spring rates with a specific displacement at a certain maximum lateral acceleration

In [17]:
import numpy as np
import pandas as pd
import pickle

# Importing Engine Data from Excel using PANDAS
cbf650r_engine_data_file_path = "C:/Users/maxwe/Downloads/FSAE/2023-2024 Car/Repo/engine_data.xlsx"
cbf650r_engine_data = pd.read_excel(cbf650r_engine_data_file_path)

# Importing Car Model from car_model using PICKLE
with open('C:/Users/maxwe/Downloads/FSAE/2023-2024 Car/Repo/73_car_model.pkl', 'rb') as f:
    car_var = pickle.load(f)

# total weight of car (minus driver) (lbm)
W_T = car_var['W_T']
# weight bias, if less than 0.5, then the rear of the car will have more weight, if more than 0.5, then the front will have more weight
W_bias = car_var['W_bias']
# length of wheelbase (in)
l = car_var['l']
# vertical center of gravity (in)
h = car_var['h']
# setting pi as a number
pi = 3.14159
# tire grip limit (G's)
A_y = car_var['tire_a']
# front track width (in)
t_f = car_var['t_f']
# rear track width (in)
t_r = car_var['t_r']
# CG to roll axis (in)
H = car_var['H']
# distance between CG and front track (in)
a = car_var['a']
# distance between CG and rear track (in)
b = car_var['b']
# front roll axis height (in)
z_rf = car_var['z_rf']
# rear roll axis height (in)
z_rr = car_var['z_rr']
# Tire rates, camper from 0-4 deg, (lb/in)
T_r = car_var['T_r']
T_r0 = T_r[0]
T_r1 = T_r[1]
T_r2 = T_r[2]
T_r3 = T_r[3]
T_r4 = T_r[4]


W_f = W_bias*W_T # lbs, weight over front wheels
W_r = (1-W_bias)*W_T# lbs, weight over rear wheels
W_1 = W_f/2 #lbs, front left
W_2 = W_f/2 #lbs, front right
W_3 = W_r/2 #lbs, rear left
W_4 = W_r/2 #lbs, rear right
MR_F = 1 # Motion Ratio for Front
MR_R = 1.1786 # Motion Ratio for Rear


K_RF = 167 # lb/in, initial ride rates so that the code can run
K_RR = 162

y_F_ideal = 0.65 # in, This is the ideal wheel travel at maximum lateral acceleration 
y_R_ideal = 0.65

y_F = 1 # in, We start with a wheel travel displacement above ideal so the while loop runs properly
y_R = 1

W_F = 0 # lb, We start with initial lateral weight transfer to initiate loop
W_R = 1

print(round(W_F, 2) != round(W_R, 2))

while y_R > y_R_ideal:

    K_rollF = K_RF*t_f**2/2 # lb*in/rad, roll rate
    K_rollR = K_RR*t_r**2/2

    freq_f = 1/(2*np.pi)*np.sqrt(K_RF*12*32.2/W_2) # natrual frequency of ride rate
    freq_r = 1/(2*np.pi)*np.sqrt(K_RR*12*32.2/W_4)

    W_F = A_y*W_T/t_f*(H*K_rollF/(K_rollF+K_rollR)+b/l*z_rf) # weight transfer for front and rear tracks
    W_R = A_y*W_T/t_r*(H*K_rollR/(K_rollF+K_rollR)+a/l*z_rr)

    y_F = W_F/K_RF # wheel travel at maximum lateral acceleration
    y_R = W_R/K_RR

    K_RF = W_F/y_F_ideal # New ride rate value
    K_RR = W_R/y_R_ideal

# Calculating Wheel Rate
K_WF = T_r1*K_RF/(T_r1 - K_RF) # Calculated at a camber angle of 1 deg
K_WR = T_r1*K_RR/(T_r1 - K_RR)

# Calculating Spring Rate -- The Numerator is equivalent to the force the spring will see when the total axle displacement
#                            in the z-direction is 1 in. The denominator is the distance that the spring will compress when 
#                            the total axle displacement is 1in in the z-direction. The Fracion in the numerator is equivalent to 
#                            the moment arm of the bell crank and pushrod over the moment arm of the bell crank and spring
K_SF = K_WF*(1.9182/3.2622)*MR_F
K_SR = K_WR*(1.9182/3.2622)*MR_R

# Printing Values in easily readable form
print(f"The front ideal ride rate is {K_RF:.5} lb/in, the rear ride rate is {K_RR:.5} lb/in\n")
print(f"The front ideal wheel rate is {K_WF:.5} lb/in, the rear wheel rate is {K_WR:.5} lb/in\n")
print(f"The front ideal spring rate is {K_SF:.5} lb/in, the rear spring rate is {K_SR:.5} lb/in\n")
print(f"the front ideal wheel travel at {A_y:.5} G's in the z-axis is {y_F:.5} in, and the rear wheel travel is {y_R:.5} in\n")
print(f"The front ideal natural frequency is {freq_f:.5} Hz, the rear natural frequency is {freq_r:.5} Hz\n")
print(f"The front ideal roll rate is {K_rollF:.5} lb*in/rad, and the rear roll rate is {K_rollR:.5} lb*in/rad\n")
print(f"The front ideal roll rate is {K_rollF*pi/180:.5} lb*in/deg, and the rear roll rate is {K_rollR*pi/180:.5} lb*in/deg\n")
print(f"The front ideal weight transfer is {W_F:.5} lb, and the rear weight transfer is {W_R:.5} lb\n\n")

# Setting the spring rates to realistic values to determine what the actual ride and roll rates will be
K_SF_real = 150 # lb/in
K_SR_real = 200 # lb/in

# Calculating actual wheel rates
K_WF_real = K_SF_real*(1/MR_F)/(1.9182/3.2622) # lb/in
K_WR_real = K_SR_real*(1/MR_R)/(1.9182/3.2622) # lb/in

# Calculating actual ride rates
K_RF_real = K_WF_real*T_r1/(K_WF_real+T_r1) # lb/in
K_RR_real = K_WR_real*T_r1/(K_WR_real+T_r1) # lb/in

# Calculating actual roll rates
K_rollF_real = K_RF_real*t_f**2/2 # lb*in/rad
K_rollR_real = K_RR_real*t_r**2/2 # lb*in/rad

# Calculating actual wheel travel
W_F_real = A_y*W_T/t_f*(H*K_rollF_real/(K_rollF_real+K_rollR_real)+b/l*z_rf) # weight transfer for front and rear tracks
W_R_real = A_y*W_T/t_r*(H*K_rollR_real/(K_rollF_real+K_rollR_real)+a/l*z_rr)

# Calculating actual wheel displacement
y_F_real = W_F_real/K_RF_real 
y_R_real = W_R_real/K_RR_real

# Calculating actual natural frequencies
freq_f_real = 1/(2*np.pi)*np.sqrt(K_RF_real*12*32.2/W_2) # natrual frequency of ride rate
freq_r_real = 1/(2*np.pi)*np.sqrt(K_RR_real*12*32.2/W_4)

# Printing Values in easily readable form
print(f"The front actual ride rate is {K_RF_real:.5} lb/in, the rear ride rate is {K_RR_real:.5} lb/in\n")
print(f"The front actual wheel rate is {K_WF_real:.5} lb/in, the rear wheel rate is {K_WR_real:.5} lb/in\n")
print(f"The front actual spring rate is {K_SF_real} lb/in, the rear spring rate is {K_SR_real} lb/in\n")
print(f"the actual front wheel travel at {A_y:.5} G's in the z-axis is {y_F_real:.5} in, and the rear wheel travel is {y_R_real:.5} in\n")
print(f"The actual front natural frequency is {freq_f_real:.5} Hz, the rear natural frequency is {freq_r_real:.5} Hz\n")
print(f"The actual front roll rate is {K_rollF_real:.5} lb*in/rad, and the rear roll rate is {K_rollR_real:.5} lb*in/rad\n")
print(f"The actual front roll rate is {K_rollF_real/12*pi/180:.5} lb*ft/deg, and the rear roll rate is {K_rollR_real/12*pi/180:.5} lb*ft/deg\n")
print(f"The actual front weight transfer is {W_F_real:.5} lb, and the rear weight transfer is {W_R_real:.5} lb\n\n")
print(f"the tire rate at 1 deg camber = {T_r1}")

True
The front ideal ride rate is 176.63 lb/in, the rear ride rate is 222.47 lb/in

The front ideal wheel rate is 232.31 lb/in, the rear wheel rate is 318.66 lb/in

The front ideal spring rate is 136.6 lb/in, the rear spring rate is 220.84 lb/in

the front ideal wheel travel at 1.4 G's in the z-axis is 0.65 in, and the rear wheel travel is 0.65 in

The front ideal natural frequency is 3.1209 Hz, the rear natural frequency is 3.5025 Hz

The front ideal roll rate is 2.3242e+05 lb*in/rad, and the rear roll rate is 2.6306e+05 lb*in/rad

The front ideal roll rate is 4056.6 lb*in/deg, and the rear roll rate is 4591.2 lb*in/deg

The front ideal weight transfer is 114.81 lb, and the rear weight transfer is 144.6 lb


The front actual ride rate is 189.5 lb/in, the rear ride rate is 207.38 lb/in

The front actual wheel rate is 255.1 lb/in, the rear wheel rate is 288.59 lb/in

The front actual spring rate is 150 lb/in, the rear spring rate is 200 lb/in

the actual front wheel travel at 1.4 G's in

In [16]:
import numpy as np
import pandas as pd
import pickle

# Importing Car Model from car_model using PICKLE
with open('C:/Users/maxwe/Downloads/FSAE/2023-2024 Car/Repo/73_car_model.pkl', 'rb') as f:
    car_var = pickle.load(f)

# total weight of car (minus driver) (lbm)
W_T = car_var['W_T']
# weight bias, if less than 0.5, then the rear of the car will have more weight, if more than 0.5, then the front will have more weight
W_bias = car_var['W_bias']
# length of wheelbase (in)
l = car_var['l']
# vertical center of gravity (in)
h = car_var['h']
# setting pi as a number
pi = 3.14159
# tire grip limit (G's)
A_y = car_var['tire_a']
# front track width (in)
t_f = car_var['t_f']
# rear track width (in)
t_r = car_var['t_r']
# CG to roll axis (in)
H = car_var['H']
# distance between CG and front track (in)
a = car_var['a']
# distance between CG and rear track (in)
b = car_var['b']
# front roll axis height (in)
z_rf = car_var['z_rf']
# rear roll axis height (in)
z_rr = car_var['z_rr']
# Tire rates, camper from 0-4 deg, (lb/in)
T_r = car_var['T_r']
T_r0 = T_r[0]
T_r1 = T_r[1]
T_r2 = T_r[2]
T_r3 = T_r[3]
T_r4 = T_r[4]
# coefficient of friction based on camber and normal force
cmbr_coeff_fy_df = car_var['cmbr_coeff_fy_df']
# Camber rate, front and rear, (deg/in)
CMB_RT_F = car_var['CMB_RT_F']
CMB_RT_R = car_var['CMB_RT_R']
# Static camber values. (deg)
CMB_STC_F = car_var['CMB_STC_F']
CMB_STC_R = car_var['CMB_STC_R']


W_f = W_bias*W_T # lbs, weight over front wheels
W_r = (1-W_bias)*W_T# lbs, weight over rear wheels
W_1 = W_f/2 #lbs, front left
W_2 = W_f/2 #lbs, front right
W_3 = W_r/2 #lbs, rear left
W_4 = W_r/2 #lbs, rear right
MR_F = car_var['MR_F'] # Motion Ratio for Front
MR_R = car_var['MR_R'] # Motion Ratio for Rear
max_jounce_f = car_var['max_jounce_f'] # in, maximum suspension displacement in jounce for front and rear
max_jounce_r = car_var['max_jounce_r']
max_droop_f = car_var['max_droop_f'] # in, maximum suspension displacement in droop for front and rear
max_droop_r = car_var['max_droop_r']

y_F_ideal = 0.65 # in, This is the ideal wheel travel at maximum lateral acceleration 
y_R_ideal = 0.65



print(round(W_F, 2) != round(W_R, 2))

properties = np.zeros((17,300**2))

for i in np.arange(1,300):
    for j in np.arange(1,300):
        K_RF = i
        K_RR = j

        K_rollF = K_RF*t_f**2/2 # lb*in/rad, roll rate
        K_rollR = K_RR*t_r**2/2

        freq_f = 1/(2*np.pi)*np.sqrt(K_RF*12*32.2/W_2) # natrual frequency of ride rate
        freq_r = 1/(2*np.pi)*np.sqrt(K_RR*12*32.2/W_4)

        a_y_G = 1 # This value initiates the maximum lateral acceleration value
        err = 1 # initiating error for the 'while' loop    
        n0 = 0
        a_y_max_lst = []
        while err > 0.01:
            n0 += 1
            change_W_y_f = a_y_G*W_T/t_f*(H*K_rollF/(K_rollF+K_rollR)+b/l*z_rf) # calculating lateral weight transfer for front track
            change_W_y_r = a_y_G*W_T/t_r*(H*K_rollR/(K_rollF+K_rollR)+a/l*z_rr) # calculating lateral weight transfer for front track
            W_in_f = W_2 - change_W_y_f # calculating normal force acting on front inside tire
            W_out_f = W_2 + change_W_y_f # calculating normal force acting on front outside tire
            W_in_r = W_4 - change_W_y_r # calculating normal force acting on rear inside tire
            W_out_r = W_4 + change_W_y_r # calculating normal force acting on rear outside tire

            # With camber and normal force known, calculate maximum possible lateral and longitudinal forces for each wheel. Then,
            # create a friction ellipse for each wheel
            disp_in_f = (W_in_f - W_2)/K_RF # calculating vertical suspension displacement
            disp_out_f = (W_out_f - W_2)/K_RF
            disp_in_r = (W_in_r - W_2)/K_RR
            disp_out_r = (W_out_r - W_2)/K_RR


            cmb_in_f = CMB_RT_F*disp_in_f + CMB_STC_F + (180/np.pi)*np.arctan((-disp_in_f+disp_out_f)/t_f) # calculating camber values, accounting for roll of frame
            cmb_out_f = CMB_RT_F*disp_out_f + CMB_STC_F - (180/np.pi)*np.arctan((-disp_in_f+disp_out_f)/t_f)
            cmb_in_r = CMB_RT_R*disp_in_r + CMB_STC_R + (180/np.pi)*np.arctan((-disp_in_r+disp_out_r)/t_r)
            cmb_out_r = CMB_RT_R*disp_out_r + CMB_STC_R - (180/np.pi)*np.arctan((-disp_in_r+disp_out_r)/t_r)

            if cmb_in_f > 4 or cmb_in_f < 0:
                break

            if cmb_out_f > 4 or cmb_out_f < 0:
                break

            if cmb_in_r > 4 or cmb_in_r < 0:
                break

            if cmb_out_r > 4 or cmb_out_r < 0:
                break

            f_y_max_in_f = W_in_f*cmbr_coeff_fy_df[int(cmb_in_f*10),int(W_in_f)]  # calculating lateral coeff. friction/max lat. G's
            f_y_max_out_f = W_out_f*cmbr_coeff_fy_df[int(cmb_out_f*10),int(W_out_f)]
            f_y_max_in_r = W_in_r*cmbr_coeff_fy_df[int(cmb_in_r*10),int(W_in_r)]
            f_y_max_out_r = W_out_r*cmbr_coeff_fy_df[int(cmb_out_r*10),int(W_out_r)]

            # Add up all the max. potential lateral forces from each wheel, and divide by the weight of the car to find lateral acceleration in Gs
            f_y_max = f_y_max_in_f + f_y_max_out_f + f_y_max_in_r + f_y_max_out_r
            new_a_y_G = f_y_max/W_T
            a_y_max_lst.append(new_a_y_G)

            if n0 > 25:
                slope_1 = (a_y_max_lst[-1]-a_y_max_lst[-20])/20 # Slope of iterative values. If the slope is approximately zero, then the values have essentially converged
                if slope_1 > 1e-3: # If the slope is more than 1e-4, then there is a considerable slope that shows that the values aren't converging
                    raise SystemExit(f"\nMaximum Longitudinal Force isn't converging!\n")
                    exit() 
                else:
                    a_y_G = np.average(a_y_max_lst[-6:-1]) # Takes the average of the last few a_y_G values and breaks the while loop
                    break            

            err = abs(new_a_y_G-a_y_G)
            a_y_G = new_a_y_G  

        max_a_y_G = a_y_G

        W_F = max_a_y_G*W_T/t_f*(H*K_rollF/(K_rollF+K_rollR)+b/l*z_rf) # weight transfer for front and rear tracks
        W_R = max_a_y_G*W_T/t_r*(H*K_rollR/(K_rollF+K_rollR)+a/l*z_rr)

        y_F = W_F/K_RF # wheel travel at maximum lateral acceleration
        y_R = W_R/K_RR

        if y_F <= 0.65*max_jounce_f and y_R <= 0.65*max_jounce_r:
            properties[0,i*j] = K_RF
            properties[1,i*j] = K_RR
            properties[2,i*j] = K_rollF
            properties[3,i*j] = K_rollR
            properties[4,i*j] = freq_f
            properties[5,i*j] = freq_r
            properties[6,i*j] = max_a_y_G
            properties[7,i*j] = W_F
            properties[8,i*j] = W_R
            properties[9,i*j] = y_F
            properties[10,i*j] = y_R
            properties[11,i*j] = (f_y_max_in_f + f_y_max_out_f)/W_T         
            properties[12,i*j] = (f_y_max_in_r + f_y_max_out_r)/W_T     
            properties[13,i*j] = cmb_in_f 
            properties[14,i*j] = cmb_out_f
            properties[15,i*j] = cmb_in_r
            properties[16,i*j] = cmb_out_r  

a_y_lst = list(properties[6,:])
K_RF = properties[0, a_y_lst.index(max(a_y_lst))]
K_RR = properties[1, a_y_lst.index(max(a_y_lst))]
K_rollF = properties[2, a_y_lst.index(max(a_y_lst))]
K_rollR = properties[3, a_y_lst.index(max(a_y_lst))]
freq_f = properties[4, a_y_lst.index(max(a_y_lst))]
freq_r = properties[5, a_y_lst.index(max(a_y_lst))]
max_a_y_G = max(a_y_lst)
W_F = properties[7, a_y_lst.index(max(a_y_lst))]
W_R = properties[8, a_y_lst.index(max(a_y_lst))]
y_F = properties[9, a_y_lst.index(max(a_y_lst))]
y_R = properties[10, a_y_lst.index(max(a_y_lst))]
a_y_F = properties[11, a_y_lst.index(max(a_y_lst))]
a_y_R = properties[12, a_y_lst.index(max(a_y_lst))]
cmb_in_f = properties[13, a_y_lst.index(max(a_y_lst))]
cmb_out_f = properties[14, a_y_lst.index(max(a_y_lst))]
cmb_in_r = properties[15, a_y_lst.index(max(a_y_lst))]
cmb_out_r = properties[16, a_y_lst.index(max(a_y_lst))]

# Calculating Wheel Rate
K_WF = T_r1*K_RF/(T_r1 - K_RF) # Calculated at a camber angle of 1 deg
K_WR = T_r1*K_RR/(T_r1 - K_RR)

# Calculating Spring Rate -- The Numerator is equivalent to the force the spring will see when the total axle displacement
#                            in the z-direction is 1 in. The denominator is the distance that the spring will compress when 
#                            the total axle displacement is 1in in the z-direction. The Fracion in the numerator is equivalent to 
#                            the moment arm of the bell crank and pushrod over the moment arm of the bell crank and spring
K_SF = K_WF*(1.9182/3.2622)*MR_F
K_SR = K_WR*(1.9182/3.2622)*MR_R

# Printing Values in easily readable form
print(f"The front ideal ride rate is {K_RF:.5} lb/in, the rear ride rate is {K_RR:.5} lb/in\n")
print(f"The front ideal wheel rate is {K_WF:.5} lb/in, the rear wheel rate is {K_WR:.5} lb/in\n")
print(f"The front ideal spring rate is {K_SF:.5} lb/in, the rear spring rate is {K_SR:.5} lb/in\n")
print(f"the front ideal wheel travel at {max_a_y_G:.5} G's in the z-axis is {y_F:.5} in, and the rear wheel travel is {y_R:.5} in\n")
print(f"The front ideal natural frequency is {freq_f:.5} Hz, the rear natural frequency is {freq_r:.5} Hz\n")
print(f"The front ideal roll rate is {K_rollF:.5} lb*in/rad, and the rear roll rate is {K_rollR:.5} lb*in/rad\n")
print(f"The front ideal roll rate is {K_rollF*pi/180:.5} lb*in/deg, and the rear roll rate is {K_rollR*pi/180:.5} lb*in/deg\n")
print(f"The front ideal weight transfer is {W_F:.5} lb, and the rear weight transfer is {W_R:.5} lb\n")
print(f"Acceleration from front track is {a_y_F} G's and the acceleration from the rear tack is {a_y_R} G's\n")
print(f"Front outside camber is {cmb_out_f} deg, front inside camber is {cmb_in_f} deg, rear outside camber is {cmb_out_r} deg, rear inside camber is {cmb_in_r} deg")

# Setting the spring rates to realistic values to determine what the actual ride and roll rates will be
K_SF_real = 150 # lb/in
K_SR_real = 200 # lb/in

# Calculating actual wheel rates
K_WF_real = K_SF_real*(1/MR_F)/(1.9182/3.2622) # lb/in
K_WR_real = K_SR_real*(1/MR_R)/(1.9182/3.2622) # lb/in

# Calculating actual ride rates
K_RF_real = K_WF_real*T_r1/(K_WF_real+T_r1) # lb/in
K_RR_real = K_WR_real*T_r1/(K_WR_real+T_r1) # lb/in

# Calculating actual roll rates
K_rollF_real = K_RF_real*t_f**2/2 # lb*in/rad
K_rollR_real = K_RR_real*t_r**2/2 # lb*in/rad

# Calculating actual wheel travel
W_F_real = A_y*W_T/t_f*(H*K_rollF_real/(K_rollF_real+K_rollR_real)+b/l*z_rf) # weight transfer for front and rear tracks
W_R_real = A_y*W_T/t_r*(H*K_rollR_real/(K_rollF_real+K_rollR_real)+a/l*z_rr)

# Calculating actual wheel displacement
y_F_real = W_F_real/K_RF_real 
y_R_real = W_R_real/K_RR_real

# Calculating actual natural frequencies
freq_f_real = 1/(2*np.pi)*np.sqrt(K_RF_real*12*32.2/W_2) # natrual frequency of ride rate
freq_r_real = 1/(2*np.pi)*np.sqrt(K_RR_real*12*32.2/W_4)

# Printing Values in easily readable form
print(f"The front actual ride rate is {K_RF_real:.5} lb/in, the rear ride rate is {K_RR_real:.5} lb/in\n")
print(f"The front actual wheel rate is {K_WF_real:.5} lb/in, the rear wheel rate is {K_WR_real:.5} lb/in\n")
print(f"The front actual spring rate is {K_SF_real} lb/in, the rear spring rate is {K_SR_real} lb/in\n")
print(f"the actual front wheel travel at {max_a_y_G:.5} G's in the z-axis is {y_F_real:.5} in, and the rear wheel travel is {y_R_real:.5} in\n")
print(f"The actual front natural frequency is {freq_f_real:.5} Hz, the rear natural frequency is {freq_r_real:.5} Hz\n")
print(f"The actual front roll rate is {K_rollF_real:.5} lb*in/rad, and the rear roll rate is {K_rollR_real:.5} lb*in/rad\n")
print(f"The actual front roll rate is {K_rollF_real/12*pi/180:.5} lb*ft/deg, and the rear roll rate is {K_rollR_real/12*pi/180:.5} lb*ft/deg\n")
print(f"The actual front weight transfer is {W_F_real:.5} lb, and the rear weight transfer is {W_R_real:.5} lb\n\n")
print(f"the tire rate at 1 deg camber = {T_r1}")







True
The front ideal ride rate is 268.0 lb/in, the rear ride rate is 179.0 lb/in

The front ideal wheel rate is 421.14 lb/in, the rear wheel rate is 236.42 lb/in

The front ideal spring rate is 247.64 lb/in, the rear spring rate is 163.85 lb/in

the front ideal wheel travel at 1.2169 G's in the z-axis is 0.47522 in, and the rear wheel travel is 0.5398 in

The front ideal natural frequency is 3.8442 Hz, the rear natural frequency is 3.1417 Hz

The front ideal roll rate is 3.5265e+05 lb*in/rad, and the rear roll rate is 2.1166e+05 lb*in/rad

The front ideal roll rate is 6154.8 lb*in/deg, and the rear roll rate is 3694.1 lb*in/deg

The front ideal weight transfer is 127.36 lb, and the rear weight transfer is 96.623 lb

Acceleration from front track is 0.6032824474054631 G's and the acceleration from the rear tack is 0.6136462088737883 G's

Front outside camber is 0.6519329246224779 deg, front inside camber is 1.348067075377522 deg, rear outside camber is 0.40384238985591536 deg, rear insi